# Handless Chess Engine

### Importing Libraries

In [1]:
import chess

In [2]:
import chess.pgn

In [3]:
import numpy as np

In [4]:
import random

In [5]:
from sklearn.model_selection import train_test_split

In [6]:
from tensorflow.keras import layers, models

### Function to convert chess notation to indexes on chess boards

In [7]:
def chess_notation_to_indices(move):
    start_square = (8 - int(move[1]), ord(move[0]) - ord('a'))
    end_square = (8 - int(move[3]), ord(move[2]) - ord('a'))
    return start_square, end_square

In [8]:
def encode_board(board):
    encoded_board = np.zeros((8, 8, 13), dtype=np.int8)
    piece_to_index = {
        'P': 0, 'N': 1, 'B': 2, 'R': 3, 'Q': 4, 'K': 5,
        'p': 6, 'n': 7, 'b': 8, 'r': 9, 'q': 10, 'k': 11,
    }
    for square in chess.SQUARES:
        piece = board.piece_at(square)
        if piece:
            r, c = divmod(square, 8)
            encoded_board[r, c, piece_to_index[piece.symbol()]] = 1
    encoded_board[:, :, 12] = board.turn
    return encoded_board

In [9]:
def generate_chess_games(num_games=500):
    games = []
    for _ in range(num_games):
        game = chess.pgn.Game()
        board = chess.Board()
        node = game
        while not board.is_game_over():
            legal_moves = list(board.legal_moves)
            if not legal_moves:
                break
            move = random.choice(legal_moves)
            board.push(move)
            node = node.add_variation(move)
        games.append(game)
    return games

In [10]:
def create_dataset(games):
    X = []
    y = []
    for game in games:
        board = game.board()
        for move in game.mainline_moves():
            if move in board.legal_moves:
                X.append(encode_board(board.copy()))
                start_square, end_square = chess_notation_to_indices(move.uci())
                y.append((start_square, end_square))
                board.push(move)
    return np.array(X), np.array(y)

In [11]:
games=generate_chess_games()

In [12]:
X, y = create_dataset(games)

In [13]:
y = np.array([np.ravel(sq) for sq in y])

In [14]:
if len(X) == 0 or len(y) == 0:
    raise ValueError("Generated dataset is empty. Check the move generation and validation process.")
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)
print(f"X_train shape: {X_train.shape}, y_train shape: {y_train.shape}")
print(f"X_test shape: {X_test.shape}, y_test shape: {y_test.shape}")

X_train shape: (131865, 8, 8, 13), y_train shape: (131865, 4)
X_test shape: (43956, 8, 8, 13), y_test shape: (43956, 4)


## Resnet Model

In [15]:
def residual_block(input_tensor, filters, kernel_size=3):
    x = layers.Conv2D(filters, kernel_size, activation='relu', padding='same')(input_tensor)
    x = layers.BatchNormalization()(x)
    x = layers.Conv2D(filters, kernel_size, activation=None, padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.add([x, input_tensor])
    x = layers.Activation('relu')(x)
    return x

def resnet_model(input_size=(8, 8, 13)):
    inputs = layers.Input(input_size)

    x = layers.Conv2D(64, 3, activation='relu', padding='same')(inputs)

    for _ in range(4):
        x = residual_block(x, 64)
    x = layers.Flatten()(x)
    x = layers.Dense(128, activation='relu')(x)
    x = layers.Dense(64, activation='relu')(x)
    outputs = layers.Dense(4)(x)
    model = models.Model(inputs=inputs, outputs=outputs)
    return model

In [16]:
model = resnet_model()
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=5, batch_size=32, validation_split=0.2)

Epoch 1/5
3297/3297 ━━━━━━━━━━━━━━━━━━━━ 320s 93ms/step - accuracy: 0.4189 - loss: 3.9492 - val_accuracy: 0.5057 - val_loss: 3.3467
Epoch 2/5
3297/3297 ━━━━━━━━━━━━━━━━━━━━ 254s 77ms/step - accuracy: 0.4701 - loss: 3.3324 - val_accuracy: 0.4739 - val_loss: 3.2954
Epoch 3/5
3297/3297 ━━━━━━━━━━━━━━━━━━━━ 249s 76ms/step - accuracy: 0.4808 - loss: 3.3002 - val_accuracy: 0.4739 - val_loss: 3.3265
Epoch 4/5
3297/3297 ━━━━━━━━━━━━━━━━━━━━ 272s 79ms/step - accuracy: 0.4778 - loss: 3.2455 - val_accuracy: 0.4652 - val_loss: 3.2397
Epoch 5/5
3297/3297 ━━━━━━━━━━━━━━━━━━━━ 259s 78ms/step - accuracy: 0.4845 - loss: 3.1994 - val_accuracy: 0.4798 - val_loss: 3.2364


In [17]:
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test loss: {loss}')
print(f'Test accuracy: {accuracy}')

1374/1374 ━━━━━━━━━━━━━━━━━━━━ 36s 27ms/step - accuracy: 0.4748 - loss: 3.2745
Test loss: 3.2645034790039062
Test accuracy: 0.4770224690437317


In [18]:
predictions = model.predict(X_test)

1374/1374 ━━━━━━━━━━━━━━━━━━━━ 37s 27ms/step


In [19]:
print('Predicted moves vs Actual moves:')

Predicted moves vs Actual moves:


In [20]:
for i in range(5):
    pred = predictions[i]
    actual = y_test[i]

    pred_start_square = (8 - int(pred[0]), chr(int(pred[1]) + ord('a')))
    pred_end_square = (8 - int(pred[2]), chr(int(pred[3]) + ord('a')))

    actual_start_square = (8 - actual[0], chr(actual[1] + ord('a')))
    actual_end_square = (8 - actual[2], chr(actual[3] + ord('a')))

    print(f'Predicted start: {pred_start_square}, Predicted end: {pred_end_square}')
    print(f'Actual start: {actual_start_square}, Actual end: {actual_end_square}')
    print()

Predicted start: (5, 'd'), Predicted end: (5, 'd')
Actual start: (4, 'd'), Actual end: (3, 'e')

Predicted start: (5, 'e'), Predicted end: (5, 'd')
Actual start: (8, 'f'), Actual end: (7, 'e')

Predicted start: (7, 'b'), Predicted end: (6, 'c')
Actual start: (6, 'b'), Actual end: (4, 'a')

Predicted start: (8, 'f'), Predicted end: (8, 'f')
Actual start: (7, 'g'), Actual end: (8, 'h')

Predicted start: (4, 'f'), Predicted end: (4, 'e')
Actual start: (2, 'f'), Actual end: (2, 'g')



In [21]:
def calculate_move_accuracy(predictions, y_test):
    correct_predictions = 0
    total_predictions = len(predictions)
    
    for pred, actual in zip(predictions, y_test):
        pred_start_square = (8 - int(pred[0]), chr(int(pred[1]) + ord('a')))
        pred_end_square = (8 - int(pred[2]), chr(int(pred[3]) + ord('a')))

        actual_start_square = (8 - actual[0], chr(actual[1] + ord('a')))
        actual_end_square = (8 - actual[2], chr(actual[3] + ord('a')))
        
        if pred_start_square == actual_start_square and pred_end_square == actual_end_square:
            correct_predictions += 1
            
    accuracy = correct_predictions / total_predictions
    return accuracy

In [22]:
move_accuracy = calculate_move_accuracy(predictions, y_test)
print(f'Move prediction accuracy on test dataset: {move_accuracy * 100:.2f}%')

Move prediction accuracy on test dataset: 0.39%


## CNN Model with LSTM Layers

In [32]:
import tensorflow as tf

In [33]:
from tensorflow.keras import layers, models
model = models.Sequential()

model.add(layers.Conv2D(64, (3, 3), activation='relu', padding='same', input_shape=(8, 8, 13)))
model.add(layers.Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(layers.BatchNormalization())
model.add(layers.MaxPooling2D(pool_size=(2, 2)))

model.add(layers.Conv2D(128, (3, 3), activation='relu', padding='same'))
model.add(layers.Conv2D(128, (3, 3), activation='relu', padding='same'))
model.add(layers.BatchNormalization())
model.add(layers.MaxPooling2D(pool_size=(2, 2)))

model.add(layers.Conv2D(256, (3, 3), activation='relu', padding='same'))
model.add(layers.Conv2D(256, (3, 3), activation='relu', padding='same'))
model.add(layers.BatchNormalization())
model.add(layers.MaxPooling2D(pool_size=(2, 2)))

model.add(layers.Reshape((-1, 256)))
model.add(layers.LSTM(256, return_sequences=True))
model.add(layers.Dropout(0.5))

model.add(layers.Flatten())
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(4))

E:\ANACONDA\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [34]:
model.compile(optimizer='adam', loss='mse', metrics=['accuracy'])
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d_11 (Conv2D)                   │ (None, 8, 8, 64)            │           7,552 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_12 (Conv2D)                   │ (None, 8, 8, 64)            │          36,928 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_8                │ (None, 8, 8, 64)            │             256 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 4, 4, 64)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_13 (Conv2D)                   │ (None, 4, 4, 128)           │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_14 (Conv2D)                   │ (None, 4, 4, 128)           │         147,584 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_9                │ (None, 4, 4, 128)           │             512 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 2, 2, 128)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_15 (Conv2D)                   │ (None, 2, 2, 256)           │         295,168 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_16 (Conv2D)                   │ (None, 2, 2, 256)           │         590,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_10               │ (None, 2, 2, 256)           │           1,024 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 1, 1, 256)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ reshape (Reshape)                    │ (None, 1, 256)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm (LSTM)                          │ (None, 1, 256)              │         525,312 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_12 (Dropout)                 │ (None, 1, 256)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_2 (Flatten)                  │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_14 (Dense)                     │ (None, 512)                 │         131,584 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_13 (Dropout)                 │ (None, 512)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_15 (Dense)                     │ (None, 256)                 │         131,328 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_14 (Dropout)                 │ (None, 256)                 │              

 Total params: 1,942,212 (7.41 MB)

 Trainable params: 1,941,316 (7.41 MB)

 Non-trainable params: 896 (3.50 KB)

In [36]:
model.fit(X_train, y_train, epochs=5, batch_size=32, validation_data=(X_test, y_test))

Epoch 1/5
4121/4121 ━━━━━━━━━━━━━━━━━━━━ 231s 49ms/step - accuracy: 0.4003 - loss: 4.3796 - val_accuracy: 0.4630 - val_loss: 3.6346
Epoch 2/5
4121/4121 ━━━━━━━━━━━━━━━━━━━━ 196s 48ms/step - accuracy: 0.4690 - loss: 3.5419 - val_accuracy: 0.5008 - val_loss: 3.4036
Epoch 3/5
4121/4121 ━━━━━━━━━━━━━━━━━━━━ 185s 45ms/step - accuracy: 0.4805 - loss: 3.4614 - val_accuracy: 0.4938 - val_loss: 3.3481
Epoch 4/5
4121/4121 ━━━━━━━━━━━━━━━━━━━━ 206s 50ms/step - accuracy: 0.4790 - loss: 3.3877 - val_accuracy: 0.4687 - val_loss: 3.3436
Epoch 5/5
4121/4121 ━━━━━━━━━━━━━━━━━━━━ 210s 51ms/step - accuracy: 0.4795 - loss: 3.3478 - val_accuracy: 0.4818 - val_loss: 3.3018


In [37]:
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Loss: {loss}, Accuracy: {accuracy}')

1374/1374 ━━━━━━━━━━━━━━━━━━━━ 31s 23ms/step - accuracy: 0.4803 - loss: 3.3127
Loss: 3.3017842769622803, Accuracy: 0.4818227291107178


In [38]:
predictions = model.predict(X_test)

1374/1374 ━━━━━━━━━━━━━━━━━━━━ 28s 20ms/step


In [39]:
print('Predicted moves vs Actual moves:')
for i in range(5):
    pred = predictions[i]
    actual = y_test[i]

    pred_start_square = (8 - int(pred[0]), chr(int(pred[1]) + ord('a')))
    pred_end_square = (8 - int(pred[2]), chr(int(pred[3]) + ord('a')))

    actual_start_square = (8 - actual[0], chr(actual[1] + ord('a')))
    actual_end_square = (8 - actual[2], chr(actual[3] + ord('a')))

    print(f'Predicted start: {pred_start_square}, Predicted end: {pred_end_square}')
    print(f'Actual start: {actual_start_square}, Actual end: {actual_end_square}')
    print()

Predicted moves vs Actual moves:
Predicted start: (5, 'd'), Predicted end: (5, 'd')
Actual start: (4, 'd'), Actual end: (3, 'e')

Predicted start: (4, 'd'), Predicted end: (4, 'd')
Actual start: (8, 'f'), Actual end: (7, 'e')

Predicted start: (7, 'c'), Predicted end: (7, 'c')
Actual start: (6, 'b'), Actual end: (4, 'a')

Predicted start: (8, 'g'), Predicted end: (7, 'f')
Actual start: (7, 'g'), Actual end: (8, 'h')

Predicted start: (4, 'e'), Predicted end: (4, 'e')
Actual start: (2, 'f'), Actual end: (2, 'g')



In [40]:
move_accuracy = calculate_move_accuracy(predictions, y_test)
print(f'Move prediction accuracy on test dataset: {move_accuracy * 100:.2f}%')

Move prediction accuracy on test dataset: 0.53%
